In [1]:
import pandas as pd
import numpy as np

In [2]:
import os

In [3]:
print(os.listdir())

['.git', '.ipynb_checkpoints', 'assets', 'pictures', 'project_group_IOT.ipynb', 'README.md']


In [4]:
os.chdir("./assets")

In [5]:
test = pd.read_csv("test.csv")
print(test.head(1))

   id  battery_power  blue  clock_speed  dual_sim  fc  four_g  int_memory  \
0   1           1043     1          1.8         1  14       0           5   

   m_dep  mobile_wt  ...  pc  px_height  px_width   ram  sc_h  sc_w  \
0    0.1        193  ...  16        226      1412  3476    12     7   

   talk_time  three_g  touch_screen  wifi  
0          2        0             1     0  

[1 rows x 21 columns]


In [6]:
train = pd.read_csv("train.csv")
print(train.head(1))

   battery_power  blue  clock_speed  dual_sim  fc  four_g  int_memory  m_dep  \
0            842     0          2.2         0   1       0           7    0.6   

   mobile_wt  n_cores  ...  px_height  px_width   ram  sc_h  sc_w  talk_time  \
0        188        2  ...         20       756  2549     9     7         19   

   three_g  touch_screen  wifi  price_range  
0        0             0     1            1  

[1 rows x 21 columns]


In [7]:
print(test.columns)
print(train.columns)

Index(['id', 'battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc',
       'four_g', 'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc',
       'px_height', 'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi'],
      dtype='object')
Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')


In [8]:
print(set(test.columns)-set(train.columns))
print(set(train.columns)-set(test.columns))
print(train["price_range"].describe())

{'id'}
{'price_range'}
count    2000.000000
mean        1.500000
std         1.118314
min         0.000000
25%         0.750000
50%         1.500000
75%         2.250000
max         3.000000
Name: price_range, dtype: float64


In [9]:
train_price_y = np.array(train["price_range"])
print((train_price_y))
data = train_price_y
shape = (data.size, data.max()+1)
one_hot_y = np.zeros(shape)
rows = np.arange(data.size)
one_hot_y[rows, data] = 1
print(one_hot_y)

[1 2 2 ... 3 0 3]
[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 ...
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]]


In [10]:
test_x = test.drop(columns=["id"])
train_x = train.drop(columns=["price_range"])

train in the wrong way

In [11]:
# doing as a linear regression
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(len(train_x.columns), input_dim=len(train_x.columns), activation='relu'))
model.add(Dense(len(train_x.columns)/2, input_dim=len(train_x.columns), activation='sigmoid'))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(train_x, one_hot_y, epochs=500, batch_size=64)

ModuleNotFoundError: No module named 'tensorflow'

In [12]:
nn_predict = model.predict(train_x)
# a = accuracy_score(pred,test)
# print('Accuracy is:', a*100)

NameError: name 'model' is not defined

In [13]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=10)
model.fit(train_x,train_price_y)

#cheat
y_model = model.predict(train_x)
from sklearn.metrics import accuracy_score
accuracy_score(train_price_y, y_model)

0.944

#Train in the right way

In [14]:
from sklearn.model_selection import train_test_split
x1,x2,y1,y2 = train_test_split(train_x,train_price_y, random_state=64, train_size=0.7)

In [15]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=50)
model.fit(x1,y1)

#predict
y_model = model.predict(x2)
from sklearn.metrics import accuracy_score
accuracy_score(y2, y_model)

0.92

In [16]:
X = train_x
y = train_price_y
from sklearn.model_selection import cross_val_score
print(cross_val_score(model, X,y,cv=5))

from sklearn.model_selection import LeaveOneOut
scores = cross_val_score(model, X,y,cv=LeaveOneOut().split(X))
print(scores.mean())

[0.93   0.9425 0.92   0.93   0.905 ]
0.928


Grid Search

In [17]:
from sklearn.model_selection import GridSearchCV
param_grid = {"n_neighbors":np.arange(1,100)}
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=7)
grid.fit(X,y)


GridSearchCV(cv=7, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

In [18]:
print(grid.best_params_)
model = grid.best_estimator_
import matplotlib.pyplot as plt

{'n_neighbors': 12}


In [19]:
y_test = model.fit(x1,y1).predict(x2)

In [20]:
accuracy_score(y2, y_test)

0.9266666666666666

In [21]:
from sklearn.svm import SVC

In [22]:
import time
import datetime

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

import matplotlib.pyplot as plt
plt.style.use('seaborn')

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_x,train_price_y, random_state=64, train_size=0.7)

In [23]:
C = 1 #@param {type:"slider", min:1.0, max:3.0, step:0.1}
kernel = "linear" #@param ['linear', 'poly', 'rbf']
gamma = "scale"  #@param ['scale', 'auto']

svm = SVC(
  C=C,
  kernel=kernel,
  gamma=gamma,
  cache_size=4096,
   random_state=0
)

svm_start_time = time.time()
model=svm.fit(X_train,y_train)
svm_end_time = time.time()
print(f"Training Time: {datetime.timedelta(seconds=svm_end_time-svm_start_time)}")

Training Time: 0:00:07.097091


In [24]:
#predict
y_model = model.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_model)

0.9683333333333334

In [25]:
# Model 1: Linear SVC version
# Create a pipeline
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
clf_linSVC = Pipeline([
    ("linear_svc", LinearSVC(C=200, loss="hinge", max_iter=100000))
])

# Train the model
clf_linSVC.fit(X_train, y_train)

# Evaluate the model's accuracy
print("Train set accuracy = " + str(clf_linSVC.score(X_train, y_train)))
print("Test set accuracy = " + str(clf_linSVC.score(X_test, y_test)))

D:\Anaconda\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Train set accuracy = 0.5364285714285715
Test set accuracy = 0.5133333333333333


In [26]:
# Model 2: Gaussian RBF Kernel version
# Create a pipeline
clf_SVC = Pipeline([
    ("linear_svc", SVC(kernel="rbf", gamma=2, C=10, max_iter=10000))
])

# Train the model
clf_SVC.fit(X_train, y_train)

# Evaluate the model's accuracy
print("Train set accuracy = " + str(clf_SVC.score(X_train, y_train)))
print("Test set accuracy = " + str(clf_SVC.score(X_test, y_test)))

Train set accuracy = 1.0
Test set accuracy = 0.23833333333333334
